In [1]:
from sklearn.metrics import roc_auc_score

import numpy as np

#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

%pylab inline
import pandas as pd
from scipy import stats
#import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from itertools import product

from sklearn import preprocessing

from sklearn.preprocessing import MinMaxScaler

import time
import datetime

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Sites 100 most popular
sites_test = np.load('sparse_matrix_Alice_dic_100_values_TEST.npy')
sites_train = np.load('sparse_matrix_Alice_dic_100_values.npy')

# 3 in row (need to be transponed)
X_train_sitelines = np.load('sor_alice_TRAIN_3inrow_appear_50times.npy')
X_test_sitelines = np.load('sor_alice_TEST_3inrow_appear_50times.npy')


# TIME
time_train = pd.read_csv('df_train_time_scaled_oct26.csv',',',  dayfirst=True)
time_test = pd.read_csv('df_test_time_scaled_oct26.csv',',',  dayfirst=True)

In [3]:
# Объединение

# TRAIN

X_f = np.hstack((sites_train, X_train_sitelines.T))

X = np.hstack((np.array(time_train), X_f))             

y = np.load('Y_TRAIN_oct24.npy')



# TEST
X_TEST_f = np.hstack((sites_test, X_test_sitelines.T))
X_TEST = np.hstack((X_TEST_f, np.array(time_test)))

In [4]:
from sklearn.cross_validation import train_test_split
# Разбиение выборки
X_train_big, X_test, y_train_big, y_test = train_test_split(X, y,
                                                  test_size = 0.2,
                                                  random_state=42)

X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_train_big,  y_train_big,
                                                  test_size = 0.2,
                                                  random_state=42)

/Users/aurinko/anaconda/envs/Py27_proekt/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Voting Ensemble for Classification

In [8]:
# Voting Ensemble for Classification
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn import ensemble , cross_validation, learning_curve, metrics, datasets, tree 


seed = 42
kfold = model_selection.KFold(n_splits=10, random_state=seed)

# create the sub models
estimators = []

model1 = DecisionTreeClassifier(random_state=42)
estimators.append(('tree', model1))


model2 = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=42),
                         algorithm="SAMME",
                         n_estimators=20, random_state=42)
estimators.append(('Ada', model2))



model3 =  GaussianNB()
estimators.append(('NB', model3))



# create the ensemble model
ensemble = VotingClassifier(estimators, voting='soft')


In [ ]:
# По ТЕСТОВОЙ ВЫБОРКЕ
ensemble.fit(X, y)

# make predictions
ensemble.predict_proba(X_TEST)

In [39]:
# Подготовка к загрузке на кегл
p = ensemble.predict_proba(X_TEST)

# Преобразования ответа

p_1 = p[:,1:]                # удаляю первый столбец  беру только столбец с ответами к классу 1
p_2 = p_1.reshape(len(p_1))  # изменяю форму на строку

# создаю полную строку сессий
session_id_full = range(1, len(p_2)+1)

print (len(session_id_full))
print (len(p_2))

#создаю датафрейм ответа
d = {'session_id': session_id_full, 'target': p_2}
a = pd.DataFrame(data=d, )
a.head()

a.to_csv('TEST_6_SUB.csv', sep=',', index=False)

82797
82797


In [9]:
ensemble.fit(X_train_sm, y_train_sm)

# make predictions
ensemble.predict_proba(X_test_sm)

array([[ 0.91035286,  0.08964714],
       [ 0.91035286,  0.08964714],
       [ 0.91035286,  0.08964714],
       ..., 
       [ 0.91035286,  0.08964714],
       [ 0.57701953,  0.42298047],
       [ 0.91035286,  0.08964714]])

In [10]:
print ('matrix_test_sm') 
print (metrics.confusion_matrix(y_test_sm, ensemble.predict(X_test_sm)))
print ("f1_score")
print ('test', metrics.f1_score(y_test_sm, ensemble.predict(X_test_sm)))
print ("")
print ('matrix_test') 
print (metrics.confusion_matrix(y_test, ensemble.predict(X_test)))
print ("f1_score")
print ('test', metrics.f1_score(y_test, ensemble.predict(X_test)))
print ("")
print ("")

matrix_test_sm
[[40111    95]
 [  122   242]]
f1_score
('test', 0.69044222539229672)

matrix_test
[[50145   110]
 [  137   321]]
f1_score
('test', 0.72215973003374578)




In [ ]:
matrix_test_sm
[[40173    33]
 [  126   238]]
f1_score
('test', 0.74960629921259847)

matrix_test
[[50217    38]
 [  156   302]]
f1_score
('test', 0.75689223057644106)

matrix_test_sm
[[40173    33]
 [  126   238]]
f1_score
('test', 0.74960629921259847)

matrix_test
[[50217    38]
 [  156   302]]
f1_score
('test', 0.75689223057644106)


model1 = DecisionTreeClassifier(random_state=42)
estimators.append(('tree', model1))


model2 = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=42, max_depth=9),
                         algorithm="SAMME",
                         n_estimators=20, random_state=42)
estimators.append(('Ada', model2))



model3 =  GaussianNB()
estimators.append(('NB', model3))

In [192]:
for clf, label in zip([model1, model2, model3, model4, ensemble], ['Tree', 'Ada', 'NB', 'GB', 'Ensemble']):
    scores = cross_val_score(clf, X_test_sm, y_test_sm, cv=10, scoring='f1')
    print("f1_score: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

f1_score: 0.33 (+/- 0.02) [Tree]
f1_score: 0.33 (+/- 0.03) [Ada]
f1_score: 0.03 (+/- 0.00) [NB]
f1_score: 0.21 (+/- 0.04) [GB]
f1_score: 0.35 (+/- 0.02) [Ensemble]


# DecisionTreeClassifier

In [11]:
from sklearn import ensemble , cross_validation, learning_curve, metrics, datasets, tree 

In [12]:
d_tree = tree.DecisionTreeClassifier(random_state=42)
d_tree.fit(X_train_sm, y_train_sm)
d_tree_pred = d_tree.predict(X_train_sm)

In [13]:
d_tree.predict(X_test_sm)

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
d_tree.predict_proba(X_test_sm)

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [15]:
print ('matrix_test') 
print (metrics.confusion_matrix(y_test_sm, d_tree.predict(X_test_sm)))
print ("")
print ('test', metrics.f1_score(y_test_sm, d_tree.predict(X_test_sm)))

print ('matrix_test') 
print (metrics.confusion_matrix(y_test, d_tree.predict(X_test)))
print ("")
print ('test', metrics.f1_score(y_test, d_tree.predict(X_test)))

matrix_test
[[40090   116]
 [  121   243]]

('test', 0.67219917012448138)
matrix_test
[[50123   132]
 [  137   321]]

('test', 0.70472008781558715)


In [ ]:
matrix_test
[[40090   116]
 [  121   243]]

('test', 0.67219917012448138)
matrix_test
[[50123   132]
 [  137   321]]

('test', 0.70472008781558715)

matrix_test
[[40078   128]
 [  111   253]]

('test', 0.67919463087248322)
matrix_test
[[50114   141]
 [  127   331]]

('test', 0.71182795698924739)

In [77]:
X.shape

(231052, 21)

# Наивный байес

In [19]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

In [20]:
modelNB = GaussianNB()

modelNB.fit(X_train_sm, y_train_sm)

modelNB.predict(X_test_sm)

array([0, 0, 0, ..., 0, 1, 0])

In [22]:
modelNB.predict_proba(X_test_sm)

array([[  1.00000000e+00,   8.35130831e-81],
       [  1.00000000e+00,   1.13670653e-72],
       [  1.00000000e+00,   8.22711143e-74],
       ..., 
       [  1.00000000e+00,   7.89218096e-73],
       [  0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   1.96680486e-63]])

In [21]:
print ('matrix_test_sm') 
print (metrics.confusion_matrix(y_test_sm, modelNB.predict(X_test_sm)))
print ("")
print ('test', metrics.f1_score(y_test_sm, modelNB.predict(X_test_sm)))
print ("")
print ('matrix_test') 
print (metrics.confusion_matrix(y_test, modelNB.predict(X_test)))
print ("")
print ('test', metrics.f1_score(y_test, modelNB.predict(X_test)))
print ("")
print ("")

matrix_test_sm
[[37611  2595]
 [  170   194]]

('test', 0.12305740564541705)

matrix_test
[[47008  3247]
 [  185   273]]

('test', 0.13725490196078433)




matrix_test_sm
[[37933  2273]
 [  199   165]]

('test', 0.11777301927194861)

matrix_test
[[47446  2809]
 [  233   225]]

('test', 0.12886597938144329)

# AdaBoost

In [16]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier




# Create and fit an AdaBoosted decision tree
bdt = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=42, max_depth=9),
                         algorithm="SAMME",
                         n_estimators=30, random_state=42)

bdt.fit(X_train_sm, y_train_sm)


AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
          learning_rate=1.0, n_estimators=30, random_state=42)

In [17]:
bdt.predict_proba(X_test_sm)

array([[ 0.63335132,  0.36664868],
       [ 0.6247909 ,  0.3752091 ],
       [ 0.61653167,  0.38346833],
       ..., 
       [ 0.62762507,  0.37237493],
       [ 0.46509844,  0.53490156],
       [ 0.63448206,  0.36551794]])

In [18]:
print ('matrix_test_sm') 
print (metrics.confusion_matrix(y_test_sm, bdt.predict(X_test_sm)))
print ("")
print ('test', metrics.f1_score(y_test_sm, bdt.predict(X_test_sm)))
print ("")
print ('matrix_test') 
print (metrics.confusion_matrix(y_test, bdt.predict(X_test)))
print ("")
print ('test', metrics.f1_score(y_test, bdt.predict(X_test)))
print ("")
print ("")

matrix_test_sm
[[40190    16]
 [  155   209]]

('test', 0.70967741935483863)

matrix_test
[[50237    18]
 [  182   276]]

('test', 0.73404255319148937)




In [ ]:
matrix_test_sm
[[40189    17]
 [  129   235]]

('test', 0.76298701298701299)

matrix_test
[[50235    20]
 [  157   301]]

('test', 0.77278562259306804)



AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
          learning_rate=1.0, n_estimators=20, random_state=42)

In [ ]:
matrix_test_sm
[[40188    18]
 [  124   240]]

('test', 0.77170418006430874)

matrix_test
[[50232    23]
 [  156   302]]

('test', 0.77139208173690932)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
          learning_rate=1.0, n_estimators=20, random_state=42)

In [ ]:
matrix_test_sm
[[40173    33]
 [  122   242]]

('test', 0.75743348982785597)

matrix_test
[[50221    34]
 [  159   299]]

('test', 0.75600505689001274)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
          learning_rate=1.0, n_estimators=20, random_state=42)

In [ ]:
matrix_test_sm
[[40177    29]
 [  143   221]]

('test', 0.71986970684039087)

matrix_test
[[50212    43]
 [  177   281]]

('test', 0.71867007672634264)


AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
          learning_rate=1.0, n_estimators=20, random_state=42)

matrix_test_sm
[[40176    30]
 [  140   224]]

('test', 0.72491909385113273)

matrix_test
[[50233    22]
 [  157   301]]

('test', 0.77080665813060179)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
          learning_rate=1.0, n_estimators=200, random_state=42)

# K-ближайших соседей

In [23]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
modelKnn = KNeighborsClassifier()

modelKnn.fit(X_train_sm, y_train_sm)

    
print ('matrix_test_sm') 
print (metrics.confusion_matrix(y_test_sm, modelKnn.predict(X_test_sm)))
print ("")
print ('test', metrics.f1_score(y_test_sm, modelKnn.predict(X_test_sm)))
print ("")
print ('matrix_test') 
print (metrics.confusion_matrix(y_test, modelKnn.predict(X_test)))
print ("")
print ('test', metrics.f1_score(y_test, modelKnn.predict(X_test)))
print ("")
print ("")

matrix_test_sm
[[36566    45]
 [  339    19]]

('test', 0.98961291893207826)

matrix_test
[[45702    74]
 [  408    27]]

('test', 0.98956958299971864)


# Лес

In [10]:
#Train The Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

class_weight=[]

# Create a random forest Classifier. By convention, clf means 'Classifier'
model_forest = RandomForestClassifier(random_state=42, n_estimators=30)#, bootstrap=True)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
model_forest.fit(X_train_sm, y_train_sm)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [11]:
print ('matrix_test') 
print (metrics.confusion_matrix(y_test_sm, model_forest.predict(X_test_sm)))
print ("")
print ('test', metrics.f1_score(y_test_sm, model_forest.predict(X_test_sm)))

print ('matrix_test') 
print (metrics.confusion_matrix(y_test, model_forest.predict(X_test)))
print ("")
print ('test', metrics.f1_score(y_test, model_forest.predict(X_test)))

matrix_test
[[40202     4]
 [  241   123]]

('test', 0.50101832993890016)
matrix_test
[[50253     2]
 [  280   178]]

('test', 0.55799373040752354)


In [ ]:
matrix_test
[[40202     4]
 [  241   123]]

('test', 0.50101832993890016)
matrix_test
[[50253     2]
 [  280   178]]

('test', 0.55799373040752354)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [ ]:
matrix_test
[[40200     6]
 [  250   114]]

('test', 0.4710743801652893)
matrix_test
[[50253     2]
 [  282   176]]

('test', 0.55345911949685533)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

# Gradient Boosting

In [159]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=42)
GBC.fit(X_train_sm, y_train_sm)
GBC.predict_proba(X_test_sm)

array([[  9.99267227e-01,   7.32772668e-04],
       [  9.43205846e-01,   5.67941544e-02],
       [  9.99199965e-01,   8.00034801e-04],
       ..., 
       [  9.99266384e-01,   7.33616396e-04],
       [  9.99906498e-01,   9.35016684e-05],
       [  9.98543175e-01,   1.45682525e-03]])

In [160]:

print ('matrix_test_sm') 
print (metrics.confusion_matrix(y_test_sm, GBC.predict(X_test_sm)))
print ("")
print ('test', metrics.f1_score(y_test_sm, GBC.predict(X_test_sm)))
print ("")
print ('matrix_test') 
print (metrics.confusion_matrix(y_test, GBC.predict(X_test)))
print ("")
print ('test', metrics.f1_score(y_test, GBC.predict(X_test)))
print ("")
print ("")

matrix_test_sm
[[36602     9]
 [  319    39]]

('test', 0.99112770158781682)

matrix_test
[[45763    13]
 [  390    45]]

('test', 0.99127913267403867)




# SVC

In [143]:
from sklearn.svm import SVC

svc = SVC(kernel='rbf', probability=True)

svc.fit(X_train_sm, y_train_sm)
svc.predict_proba(X_test_sm)

array([[ 0.98860499,  0.01139501],
       [ 0.9915993 ,  0.0084007 ],
       [ 0.9896708 ,  0.0103292 ],
       ..., 
       [ 0.9904273 ,  0.0095727 ],
       [ 0.9669431 ,  0.0330569 ],
       [ 0.98238038,  0.01761962]])

In [144]:

print ('matrix_test_sm') 
print (metrics.confusion_matrix(y_test_sm, svc.predict(X_test_sm)))
print ("")
print ('test', metrics.accuracy_score(y_test_sm, svc.predict(X_test_sm)))
print ("")
print ('matrix_test') 
print (metrics.confusion_matrix(y_test, svc.predict(X_test)))
print ("")
print ('test', metrics.accuracy_score(y_test, svc.predict(X_test)))
print ("")
print ("")

matrix_test_sm
[[36611     0]
 [  358     0]]

('test', 0.99031621087938548)

matrix_test
[[45776     0]
 [  435     0]]

('test', 0.99058665685659264)




# BaggingClassifier

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier

seed = 42
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = tree.DecisionTreeClassifier(random_state=seed)
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train_sm, y_train_sm, cv=kfold, scoring = 'f1')
print(results.mean())

In [ ]:
model.fit(X_train_sm, y_train_sm)

# make predictions
model.predict_proba(X_test_sm)

In [ ]:
print ('matrix_test_sm') 
print (metrics.confusion_matrix(y_test_sm, model.predict(X_test_sm)))
print ("")
print ("f1_score")

print ('test_sm', metrics.f1_score(y_test_sm, model.predict(X_test_sm)))
print ("")
print ("")
print ('matrix_test') 
print (metrics.confusion_matrix(y_test, model.predict(X_test)))
print ("")
print ("f1_score")
print ('test', metrics.f1_score(y_test, model.predict(X_test)))
print ("")
print ("")

In [ ]:
matrix_test_sm
[[36598    13]
 [  299    59]]
f1_score

('test', 0.2744186046511628)


matrix_test
[[45757    19]
 [  367    68]]

f1_score
('test', 0.26053639846743293)

# LogisticRegression

In [65]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [66]:
modelLR = LogisticRegression(random_state=42)

modelLR.fit(X_train_sm, y_train_sm)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [67]:
metrics.accuracy_score(y_test_sm, modelLR.predict(X_test_sm))

0.99031621087938548

In [68]:
metrics.accuracy_score(y_test, modelLR.predict(X_test))

0.99058665685659264

In [69]:
metrics.confusion_matrix(y_test_sm, modelLR.predict(X_test_sm))

array([[36611,     0],
       [  358,     0]])

In [70]:
metrics.confusion_matrix(y_test, modelLR.predict(X_test))

array([[45776,     0],
       [  435,     0]])